In [1]:
from pathlib import Path
import numpy as np
import os, sys
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from sql_setup import session

In [7]:
# person_id = 2 = beth
prep_id = 'DK52'
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]
    print(structure,row.x,row.y, row.section, row.person_id, row.input_type)

10N_L 45542.0 17755.0 218.0 2 manual
10N_R 45714.0 17685.0 247.0 2 manual
12N 45596.0 18540.0 234.0 2 manual
3N_L 34961.0 16285.0 232.0 2 manual
3N_R 34749.0 16231.0 243.0 2 manual
4N_L 36247.0 16030.0 223.0 2 manual
4N_R 36247.0 16056.0 247.0 2 manual
5N_L 38765.0 19548.0 172.0 2 manual
5N_R 38914.0 18314.0 309.0 2 manual
6N_L 40686.0 19124.0 257.0 2 manual
6N_R 40599.0 19425.0 220.0 2 manual
7N_L 44309.0 20809.0 307.0 2 manual
7N_R 42465.0 22042.0 309.0 2 manual
AP 46190.0 16965.0 230.0 2 manual
Amb_L 44094.0 21914.0 172.0 2 manual
DC_L 41986.0 17520.0 130.0 2 manual
DC_R 42227.0 16113.0 343.0 2 manual
IC 37176.0 9511.0 222.0 2 manual
LC_L 39808.0 16939.0 188.0 2 manual
LC_R 39934.0 16233.0 278.0 2 manual
LRt_L 47708.0 22042.0 183.0 2 manual
LRt_R 47409.0 21332.0 296.0 2 manual
PBG_L 35560.0 15975.0 138.0 2 manual
PBG_R 35814.0 15440.0 330.0 2 manual
Pn_L 35794.0 24118.0 220.0 2 manual
Pn_R 35751.0 23661.0 284.0 2 manual
RtTg 36771.0 22290.0 249.0 2 manual
SC 33868.0 9061.0 222.0 2 m